# Data Analysis 2 - ECBS5142 - Assignment 1

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
# import libs
import os
import sys
import warnings

import numpy as np
import pandas as pd
from mizani.formatters import percent_format
from plotnine import *
from datetime import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import norm
from IPython.core.display import HTML
from stargazer.stargazer import Stargazer
import statsmodels.nonparametric.kernel_regression as loess

from mizani.transforms import log_trans
from mizani.formatters import percent_format
from mizani.formatters import log_format

warnings.filterwarnings("ignore")

In [3]:
# read the data from the csv file
all_df = pd.read_csv('morg-2014-emp.csv')
all_df.head()

,Unnamed: 0,hhid,intmonth,stfips,weight,earnwke,uhours,grade92,race,ethnic,...,ownchild,chldpres,prcitshp,state,ind02,occ2012,class,unionmme,unioncov,lfsr94
0,3,2600310997690,January,AL,3151.6801,1692.00,40,43,1,NaN,...,0,0,"Native, Born In US",63,Employment services (5613),630,"Private, For Profit",No,No,Employed-At Work
1,5,75680310997590,January,AL,3457.1138,450.00,40,41,2,NaN,...,2,6,"Native, Born In US",63,Outpatient care centers (6214),5400,"Private, For Profit",No,No,Employed-Absent
2,6,75680310997590,January,AL,3936.9110,1090.00,60,41,2,NaN,...,2,6,"Native, Born In US",63,Motor vehicles and motor vehicle equipment man...,8140,"Private, For Profit",No,No,Employed-At Work
3,10,179140131100930,January,AL,3288.3640,769.23,40,40,1,NaN,...,2,4,"Native, Born In US",63,"**Publishing, except newspapers and software (...",8255,"Private, For Profit",Yes,NaN,Employed-At Work
4,11,179140131100930,January,AL,3422.8500,826.92,40,43,1,NaN,...,2,4,"Native, Born In US",63,"Banking and related activities (521, 52211,52219)",5940,"Private, For Profit",No,No,Employed-At Work


In [4]:
# Filter the data for occ2012 between 1010 and 1107
comp_sample = all_df[(all_df['occ2012'] >= 1010) & (all_df['occ2012'] <= 1107)]

#drop the all_df
del(all_df)

In [5]:
# Add a column 'hourly_wage' to the DataFrame
comp_sample['hourly_wage'] = comp_sample['earnwke'] / comp_sample['uhours']
np.round(comp_sample['hourly_wage'], decimals=2)

155       33.65
293       62.50
296       62.50
324       27.78
372       25.00
          ...  
149003    34.11
149060    41.83
149074    40.70
149137    40.87
149221    28.85
Name: hourly_wage, Length: 3782, dtype: float64

In [6]:
# Transform the 'sex' column
comp_sample['female'] = comp_sample['sex'].apply(lambda x: 1 if x == 2 else 0)

In [7]:
comp_sample.head()

,Unnamed: 0,hhid,intmonth,stfips,weight,earnwke,uhours,grade92,race,ethnic,...,prcitshp,state,ind02,occ2012,class,unionmme,unioncov,lfsr94,hourly_wage,female
155,346,673032906039520,January,AK,411.5571,1346.15,40,39,3,NaN,...,"Native, Born In US",94,Executive offices and legislative bodies (9211...,1030,Government - Local,No,No,Employed-At Work,33.653750,1
293,651,207004430306994,January,AZ,3410.8853,2500.00,40,44,4,NaN,...,"Foreign Born, US Cit By Naturalization",86,Computer systems design and related services (...,1020,"Private, For Profit",No,No,Employed-At Work,62.500000,0
296,657,236096309400800,January,AZ,3916.3279,2500.00,40,43,4,NaN,...,"Foreign Born, US Cit By Naturalization",86,Business support services (5614),1020,"Private, For Profit",No,No,Employed-At Work,62.500000,0
324,724,914299270769003,January,AZ,5115.4707,1250.00,45,43,1,NaN,...,"Native, Born In US",86,Computer systems design and related services (...,1020,"Private, For Profit",No,No,Employed-At Work,27.777778,0
372,812,498060133740930,January,AZ,2685.7836,1250.00,50,43,2,NaN,...,"Native, Born In US",86,"Colleges and universities, including junior co...",1050,Government - State,No,No,Employed-At Work,25.000000,0


In [8]:
# Map the education levels based on 'grade92'
edu_level_dict = {
    31: 'Less than 1st grade',
    32: '1st - 4th grade',
    33: '5th or 6th',
    34: '7th or 8th',
    35: '9th',
    36: '10th',
    37: '11th',
    38: '12th grade NO DIPLOMA',
    39: 'High school graduate, diploma or GED',
    40: 'Some college but no degree',
    41: 'Associate degree -- occupational/vocational',
    42: 'Associate degree -- academic program',
    43: "Bachelor's degree",
    44: "Master's degree",
    45: 'Professional school degree',
    46: 'Doctorate degree'
}
comp_sample['edu_level'] = comp_sample['grade92'].map(edu_level_dict)

In [9]:
# Add the ln_wage column
comp_sample['ln_wage'] = np.log(comp_sample['hourly_wage'])

In [10]:
# Describe the wages
comp_sample.loc[:, ["earnwke", "uhours", "hourly_wage", "ln_wage"]].describe()

,earnwke,uhours,hourly_wage,ln_wage
count,3782.000000,3782.000000,3782.000000,3782.000000
mean,1410.689482,40.993125,34.165624,3.408056
std,686.268740,6.669034,15.954855,0.532351
min,2.000000,1.000000,0.050000,-2.995732
25%,875.000000,40.000000,21.634500,3.074289
50%,1344.000000,40.000000,31.765375,3.458376
75%,1903.840000,40.000000,44.987938,3.806394
max,2884.610000,80.000000,120.192083,4.789091


## The unconditional gender gap

Here we will calculate the mean hourly wage of male and female and see the the difference in their hourly wage

In [11]:
mean_hourly_wage = comp_sample.groupby('female')['hourly_wage'].mean()
print(f"Average male's hourly wage: {mean_hourly_wage[0]}")
print(f"Average female's hourly wage: {mean_hourly_wage[1]}")
print(f"Unconditional gender wage gap between male and female: {mean_hourly_wage[0] - mean_hourly_wage[1]}")

Average male's hourly wage: 35.30327285656748
Average female's hourly wage: 30.538503930255636
Unconditional gender wage gap between male and female: 4.764768926311845


## 

In [12]:
comp_sample["female"] = comp_sample["female"].astype(int)  #from boolean to 0,1
comp_sample["female"].value_counts()

female
0    2879
1     903
Name: count, dtype: int64

In [13]:
reg1 = smf.ols(formula="ln_wage~female", data=comp_sample).fit()
reg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                ln_wage   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     69.47
Date:                Fri, 17 Nov 2023   Prob (F-statistic):           1.08e-16
Time:                        22:40:46   Log-Likelihood:                -2947.1
No. Observations:                3782   AIC:                             5898.
Df Residuals:                    3780   BIC:                             5911.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.4481      0.010    350.671      0.000       3.429       3.467
female        -0.1677      0.020     -8.335      0.000      -0.207      -0.128
==============================================================================
Omnibus:                     1081.642   Durbin-Watson:                   1.898
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8397.922
Skew:                          -1.144   Prob(JB):                         0.00
Kurtosis:                       9.932   Cond. No.                         2.51
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""